# **Liquidating a loan position**

##### The liquidation process for loan positions in the Halalend protocol is fully automated and managed by a component known as the Batcher. This component has multiple responsibilities, including processing Collateralized Debt Positions (CDPs), managing repayments, and overseeing the addition of liquidity. A key function of the Batcher is to monitor loan positions vigilantly. It automatically identifies any loan positions that have fallen below the protocol's liquidation threshold or those that have exceeded the set loan term limit. Upon detecting such cases, the Batcher proceeds with the necessary steps to secure the loan positions, ensuring the stability and security of the protocol's financial operations.

## Qualification Criteria for liquidation:

- **Collateral Ratio:** This will be compared to the liquidation theshold. If it's below then the loan position is liquidated.
- **Fees:** Loan positions are subject to liquidation fees. 

## Key Protocol Parameters:

- **Liquidation Threshold:** Determines the required overcollateralization for loan approval.
- **Liquidation Fee:** A fee charged for using the Halalend Protocol to create a loan position.
- **Loan Term:** Specifies how long the loan will last before it needs to be repaid.

## On-chain Validation:

- **Lending Validator:** A smart contract on Cardano that validates liquidation requests and ensures they comply with protocol rules.
- **Protocol Parameters UTXO:** Stores the protocol's operational parameters and lending criteria for reference in transactions.
- **Oracle/DEX Liquidity Pool UTXO:** Provides current exchange rates for collateral and loan assets, to accurately check if the loan position is below the liquidation threshold.
- **Collateral Validator:** A smart contract on Cardano that validates if the collateral utxo can be unlocked.


## Off-chain Infrastructure:

- **Batcher:** An essential background service that batches transactions for processing. This component is crucial due to Cardano's UTXO model, where a UTXO can only be consumed by one transaction in a block. The batcher ensures multiple transactions can be processed efficiently.

### Assuming the following Protocol Parameters:

In [17]:
let LIQUIDATION_THRESHOLD = 1.2
let LIQUIDATION_FEE = 10.0
let LOAN_TERM = 14 * 24 * 60 * 60 * 1000   // 14 days in milliseconds

### And the following calculations:

##### *Note: these are simplified calculations*

In [18]:
open System.Collections.Generic

// Calculates the current collateral ratio of a loan position given the exchange rates of the collateral and the loan
let calc_collateral_ratio total_collateral_value_in_usd total_loan_amount_in_usd = total_collateral_value_in_usd / total_loan_amount_in_usd

// Function to calculate total value for a single asset
let calc_collateral_asset_value (asset: IDictionary<string, obj>) =
    let exchangeRate = unbox<float>(asset.["exchange_rate"])
    let amount = unbox<float>(asset.["amount"])
    exchangeRate * amount

// Function to check for liquidation
let check_liquidation total_collateral_value_in_usd total_loan_amount_in_usd elapsed_time_in_ms =
    let collateral_ratio = calc_collateral_ratio total_collateral_value_in_usd total_loan_amount_in_usd

    printf "Collateral ratio: %f\n" collateral_ratio
    printf "Liquidataion threshold: %f\n\n" LIQUIDATION_THRESHOLD

    if collateral_ratio < LIQUIDATION_THRESHOLD || elapsed_time_in_ms > LOAN_TERM   then
        true
    else
        false
    

### Now we need to define the loan parameters and current exchange rates and check for eligibility for liquidation

In [19]:
// These are the collaterals sent by the borrower to the contract
let collateral_assets = 
    [("iUSD", dict [("exchange_rate", box 1.0); ("amount", box 1000.0)])
     ("halalend", dict [("exchange_rate", box 0.4); ("amount", box 1000.0)])
     ("hosky", dict [("exchange_rate", box 0.001); ("amount", box 100000.0)])]
    |> Map.ofList

let elapsed_time = 13 * 24 * 60 * 60 * 1000   // 14 days in milliseconds
let collateral_asset_list = collateral_assets |> Map.toList |> List.map fst
let total_collateral_value_usd = 
    collateral_assets
    |> Map.map (fun _ asset -> calc_collateral_asset_value asset)  // Calculate total for each asset
    |> Map.fold (fun acc _ value -> acc + value) 0.0       // Sum up all totals
let loan_usd = 1000.0 // this is the amount of the loan in USD

printf "Total collateral value: %f\n" total_collateral_value_usd
printf "Loan amount: %f\n\n" loan_usd

let is_liquidatable = check_liquidation total_collateral_value_usd loan_usd elapsed_time

if is_liquidatable then
    printf "The loan position is liquidatable\n"
else
    printf "The loan position is not liquidatable\n"


Total collateral value: 1500.000000
Loan amount: 1000.000000

Collateral ratio: 1.500000
Liquidataion threshold: 1.200000

The loan position is not liquidatable
